# Vitessce Data Preparation Tutorial

# Export data to AWS S3

## 1. Import dependencies

We need to import the classes and functions that we will be using from the corresponding packages.

In [ ]:
import os
import boto3
import json
from urllib.parse import quote_plus
from os.path import join
from urllib.request import urlretrieve
from anndata import read_h5ad
import scanpy as sc

from vitessce import (
    VitessceWidget,
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    AnnDataWrapper,
)

## 2. Download and process data

For this example, we need to download a dataset from the COVID-19 Cell Atlas https://www.covid19cellatlas.org/index.healthy.html#habib17.

In [ ]:
os.makedirs("data", exist_ok=True)
adata_filepath = join("data", "habib17.processed.h5ad")
urlretrieve('https://covid19.cog.sanger.ac.uk/habib17.processed.h5ad', adata_filepath)

adata = read_h5ad(adata_filepath)
top_dispersion = adata.var["dispersions_norm"][
    sorted(
        range(len(adata.var["dispersions_norm"])),
        key=lambda k: adata.var["dispersions_norm"][k],
    )[-51:][0]
]
adata.var["top_highly_variable"] = (
    adata.var["dispersions_norm"] > top_dispersion
)

## 3. Create the Vitessce configuration

Set up the configuration by adding the views and datasets of interest.

In [ ]:
vc = VitessceConfig(name='Habib et al', description='COVID-19 Healthy Donor Brain')
dataset = vc.add_dataset(name='Brain').add_object(AnnDataWrapper(
        adata,
        mappings_obsm=["X_umap"],
        mappings_obsm_names=["UMAP"],
        cell_set_obs=["CellType"],
        cell_set_obs_names=["Cell Type"],
        expression_matrix="X",
        matrix_gene_var_filter="top_highly_variable"
))
scatterplot = vc.add_view(dataset, cm.SCATTERPLOT, mapping="X_umap")
cell_sets = vc.add_view(dataset, cm.CELL_SETS)
genes = vc.add_view(dataset, cm.GENES)
heatmap = vc.add_view(dataset, cm.HEATMAP)
vc.layout((scatterplot | (cell_sets / genes)) / heatmap);

## 4. Create a `boto3` resource with S3 credentials

In [ ]:
s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=os.environ['VITESSCE_S3_ACCESS_KEY_ID'],
    aws_secret_access_key=os.environ['VITESSCE_S3_SECRET_ACCESS_KEY'],
)

## 5. Upload files to S3

The `.export(to='S3')` method on the view config instance will upload all data objects to the specified bucket. Then, the processed view config will be returned as a `dict`, with the file URLs filled in, pointing to the S3 bucket files. For more information about configuring the S3 bucket so that files are accessible over the internet, visit the "Hosting Data" page of our core documentation site.

In [ ]:
config_dict = vc.export(to='S3', s3=s3, bucket_name='vitessce-export-examples', prefix='test')

## 6. View on vitessce.io

The returned view config dict can be converted to a URL, and can be used to share the interactive visualizations with colleagues.

In [ ]:
vitessce_url = f"http://vitessce.io/?url=data:," + quote_plus(json.dumps(config_dict))
print(vitessce_url)